# Step 4 - Analysis of bicycle network results
## Project: Growing Urban Bicycle Networks

This notebook takes the existing infrastructure, the results from 03_poi_based_generation and calculates/analyzes a number of measures:
* cost (length)
* coverage  
* directness  
* efficiency
* overlap with existing networks

Contact: Michael Szell (michael.szell@gmail.com)  
Created: 2020-07-08  
Last modified: 2021-11-22

## Preliminaries

### Parameters

In [1]:
debug = False # If True, will produce plots and/or verbose output to double-check
rerun_existing = False # If True, will re-run the costly analysis of existing infra even if files already exist.
%run -i "../parameters/parameters.py"

Loaded parameters.



### Setup

In [2]:
%run -i path.py
#%run -i setup.py
%run -i setupCPH.py
if not debug: # Only do this if sure the code is bug-free!
    warnings.filterwarnings('ignore')

%load_ext watermark
%watermark -n -v -m -g -iv

Loaded PATH.

Setup finished.

Python implementation: CPython
Python version       : 3.8.2
IPython version      : 8.5.0

Compiler    : Clang 9.0.1 
OS          : Darwin
Release     : 18.7.0
Machine     : x86_64
Processor   : i386
CPU cores   : 8
Architecture: 64bit

Git hash: 91b7c308c5580dfbed9ad0603785edcccf3d2214

igraph    : 0.9.1
sklearn   : 1.1.2
geojson   : 2.5.0
geopandas : 0.11.1
csv       : 1.0
shapely   : 1.8.4
networkx  : 2.8.6
watermark : 2.3.1
osgeo     : 3.2.1
pandas    : 1.4.4
fiona     : 1.8.21
pyproj    : 3.4.0
numpy     : 1.23.3
sys       : 3.8.2 | packaged by conda-forge | (default, Apr 24 2020, 07:56:27) 
[Clang 9.0.1 ]
osmnx     : 0.16.2
matplotlib: 3.6.0



### Functions

In [3]:
%run -i functions.py

Loaded functions.



# Join attributes on all networks

In [ ]:
with open("../../bikenwgrowth_external/data/copenhagen/bikedata/mean_length_attr.pkl", 'rb') as f:
    unassigned_length_attr = pickle.load(f)
with open("../../bikenwgrowth_external/data/copenhagen/bikedata/mean_pop_den_attr.pkl", 'rb') as f:
    unassigned_pop_den_attr = pickle.load(f)
with open("../../bikenwgrowth_external/data/copenhagen/bikedata/mean_bcount_attr.pkl", 'rb') as q:
    unassigned_bcount_attr = pickle.load(q)
with open("../../bikenwgrowth_external/data/copenhagen/bikedata/mean_bcount_pop_den_att.pkl", 'rb') as q:
    unassigned_bcount_pop_den_attr = pickle.load(q)

## Join new attributes to all networks on geometry

In [ ]:
with zipfile.ZipFile("../../bikenwgrowth_external/data/copenhagen/copenhagen_carall_edges.zip", 'r') as zip_ref:
    zip_ref.extractall("../../bikenwgrowth_external/data/copenhagen/")

carall_edges = pd.read_csv("../../bikenwgrowth_external/data/copenhagen/copenhagen_carall_edges.csv")

for network in networktypesdata:
    print(network)
    
    with zipfile.ZipFile("../../bikenwgrowth_external/data/copenhagen/copenhagen_"+network+"_edges.zip", 'r') as zip_ref:
        zip_ref.extractall("../../bikenwgrowth_external/data/copenhagen/")

    biketrack_edges = pd.read_csv("../../bikenwgrowth_external/data/copenhagen/copenhagen_"+network+"_edges.csv")


    #the initial edges are joined with the new generalised counts
    result = pd.merge(biketrack_edges, carall_edges[['geometry','length_attr','bcount_attr','pop_den_attr','bcount_pop_den_attr']],how = 'left',on='geometry')
    #left_on=['u','v'], right_on = ['u','v']

    for i in range(len(result)):
        length_attr = result['length_attr'].iloc[i]
        bcount_attr = result['bcount_attr'].iloc[i]
        pop_den_attr = result['pop_den_attr'].iloc[i]
        bcount_pop_den_attr = result['bcount_pop_den_attr'].iloc[i]
        #if num is NaN
        if length_attr != length_attr:
            result['length_attr'].iloc[i]= unassigned_length_attr
        if bcount_attr != bcount_attr:
            result['bcount_attr'].iloc[i]= unassigned_bcount_attr
        if pop_den_attr != pop_den_attr:
            result['pop_den_attr'].iloc[i]= unassigned_pop_den_attr
        if bcount_pop_den_attr != bcount_pop_den_attr:
            result['bcount_pop_den_attr'].iloc[i]= unassigned_bcount_pop_den_attr
            
    result.to_csv("../../bikenwgrowth_external/data/copenhagen/copenhagen_"+network+"_edges.csv")
    compress_file("../../bikenwgrowth_external/data/copenhagen/","copenhagen_"+network+"_edges")
    


## Analyze existing infrastructure

In [ ]:
for placeid, placeinfo in tqdm(cities.items(), desc = "Cities"):
    print(placeid + ": Analyzing existing infrastructure.")
    
    # output_place is one static file for the existing city. This can be compared to the generated infrastructure.
    # Make a check if this file was already generated - it only needs to be done once. If not, generate it:
    filename = placeid + "_existing.csv"
    if rerun_existing or not os.path.isfile(PATH["results"] + placeid + "/" + filename):
        empty_metrics = {
                         "length":0,
                         "length_lcc":0,
                         "coverage": 0,
                         "directness": 0,
                         "directness_lcc": 0,
                         "poi_coverage": 0,
                         "components": 0,
                         "efficiency_global": 0,
                         #"efficiency_local": 0,
                         "efficiency_global_routed": 0,
                         #"efficiency_local_routed": 0,
                         "directness_lcc_linkwise": 0,
                         "directness_all_linkwise": 0
                        }
        output_place = {}
        for networktype in networktypes:
            output_place[networktype] = copy.deepcopy(empty_metrics)

        # Analyze all networks     
        Gs = {}
        for networktype in networktypes:
            if networktype != "biketrack_onstreet" and networktype != "bikeable_offstreet":
                Gs[networktype] = csv_to_ig(PATH["data"] + placeid + "/", placeid, networktype)
                Gs[networktype + "_simplified"] = csv_to_ig(PATH["data"] + placeid + "/", placeid, networktype + "_simplified")
                #Gs[networktype] = csv_to_ig_custom(PATH["data"] + placeid + "/", placeid, networktype)
                #Gs[networktype + "_simplified"] = csv_to_ig_custom(PATH["data"] + placeid + "/", placeid, networktype + "_simplified")
            elif networktype == "biketrack_onstreet":
                Gs[networktype] = intersect_igraphs(Gs["biketrack"], Gs["carall"])
                Gs[networktype + "_simplified"] = intersect_igraphs(Gs["biketrack_simplified"], Gs["carall_simplified"])
            elif networktype == "bikeable_offstreet":
                G_temp = copy.deepcopy(Gs["bikeable"])
                delete_overlaps(G_temp, Gs["carall"])
                Gs[networktype] = G_temp
                G_temp = copy.deepcopy(Gs["bikeable_simplified"])
                delete_overlaps(G_temp, Gs["carall_simplified"])
                Gs[networktype + "_simplified"] = G_temp
        
        with open(PATH["data"] + placeid + "/" + placeid + '_poi_' + poi_source + '_nnidscarall.csv') as f:
            nnids = [int(line.rstrip()) for line in f]

            
        covs = {}
        for networktype in tqdm(networktypes, desc = "Networks", leave = False):
            if debug: print(placeid + ": Analyzing results: " + networktype)
            metrics, cov = calculate_metrics(Gs[networktype], Gs[networktype + "_simplified"], Gs['carall'], nnids, empty_metrics, buffer_walk, numnodepairs, debug)
            #metrics, cov = calculate_metrics_custom(Gs[networktype], Gs[networktype + "_simplified"], Gs['carall'], nnids, empty_metrics, buffer_walk, numnodepairs, debug)
            
            for key, val in metrics.items():
                output_place[networktype][key] = val
            covs[networktype] = cov
        # Save the covers
        write_result(covs, "pickle", placeid, "", "", "existing_covers.pickle")
        
        # Write to CSV
        write_result(output_place, "dictnested", placeid, "", "", "existing.csv", empty_metrics)

## Analyze POI based results

In [5]:


for placeid, placeinfo in tqdm(cities.items(), desc = "Cities"):
    print(placeid + ": Analyzing results")

# Load networks
    
        
        
        
    #filename = placeid + '_poi_' + poi_source + "_" + prune_measure
    #resultfile = open(PATH["results"] + placeid + "/" + filename + ".pickle",'rb')
    #res = pickle.load(resultfile)
    #resultfile.close()
    #if debug: pp.pprint(res)
    #output, covs = calculate_metrics_additively(res["GTs"], res["GT_abstracts"], res["prune_quantiles"], G_carall, nnids, buffer_walk, numnodepairs, debug, True, Gexisting)
    #output_MST, cov_MST = calculate_metrics(res["MST"], res["MST_abstract"], G_carall, nnids, output, buffer_walk, numnodepairs, debug, True, ig.Graph(), Polygon(), False, Gexisting)
    #write_result(covs, "pickle", placeid, poi_source, prune_measure, "_covers.pickle")
    #write_result(cov_MST, "pickle", placeid, poi_source, prune_measure, "_cover_mst.pickle")
    #write_result(output, "dict", placeid, poi_source, prune_measure, ".csv")
    #write_result(output_MST, "dict", placeid, poi_source, "", "_mst.csv")
    
    
    
    
    for attr in attrlist:
        print("attr: " + attr)
        
        G_carall = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'carall')
        #G_carall = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'carall')
        Gexisting = {}
        for networktype in ["biketrack", "bikeable"]:
            Gexisting[networktype] = csv_to_ig(PATH["data"] + placeid + "/", placeid, networktype)



    # Load POIs
        with open(PATH["data"] + placeid + "/" + placeid + '_poi_' + poi_source + '_nnidscarall.csv') as f:
            nnids = [int(line.rstrip()) for line in f]
    # Load results
        filename = placeid + '_poi_' + poi_source + "_" + prune_measure + "_"+ attr
        resultfile = open(PATH["results"] + placeid + "/" + filename  +".pickle",'rb')
        res = pickle.load(resultfile)
        resultfile.close()
        if debug: pp.pprint(res)
        # Calculate
        # output contains lists for all the prune_quantile values of the corresponding results
        #output, covs = calculate_metrics_additively_custom(res["GTs"], res["GT_abstracts"], res["prune_quantiles"], G_carall, nnids, buffer_walk, numnodepairs, debug, True, Gexisting)
        #output_MST, cov_MST = calculate_metrics_custom(res["MST"], res["MST_abstract"], G_carall, nnids, output, buffer_walk, numnodepairs, debug, True, ig.Graph(), Polygon(), False, Gexisting)

        output, covs = calculate_metrics_additively(res["GTs"], res["GT_abstracts"], res["prune_quantiles"], G_carall, nnids, buffer_walk, numnodepairs, debug, True, Gexisting)
        output_MST, cov_MST = calculate_metrics(res["MST"], res["MST_abstract"], G_carall, nnids, output, buffer_walk, numnodepairs, debug, True, ig.Graph(), Polygon(), False, Gexisting)

        # Save the covers
        write_result(covs, "pickle", placeid, poi_source, prune_measure,  "_"+ attr+"_covers.pickle")
    #     write_result(covs_carminusbike, "pickle", placeid, poi_source, prune_measure, "_covers_carminusbike.pickle")
        write_result(cov_MST, "pickle", placeid, poi_source, prune_measure, "_"+ attr+"_cover_mst.pickle")

        # Write to CSV
        write_result(output, "dict", placeid, poi_source, prune_measure, "_"+ attr+".csv")
    #     write_result(output_carminusbike, "dict", placeid, poi_source, prune_measure, "_carminusbike.csv")
    #     write_result(output_carconstrictedbike, "dict", placeid, poi_source, prune_measure, "_carconstrictedbike.csv")
        write_result(output_MST, "dict", placeid, poi_source, "",  attr+"_mst.csv")
        covs.clear()
        output.clear()
        output_MST.clear()
        Gexisting.clear()

Cities:   0%|          | 0/1 [00:00<?, ?it/s]

copenhagen: Analyzing results
attr: length


Bicycle networks:   0%|          | 0/40 [00:00<?, ?it/s]

40
